In [1]:
import os
from shapely.geometry import Polygon
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from osgeo import gdal, osr
import rasterio
import geopandas as gpd
import rioxarray as rxr
import multiprocess as mp
import shapely
from shapely.ops import unary_union
%matplotlib inline

os.chdir('/Users/jmccarty/GitHub/22spring_templatematching_carto')
n_cores = 6
chunk_size = 10000

### Creating Shapes
----

In this notebook you can find a script to process an image file (a binary mask) into a set of polygons stored in a geojson. The script takes the mask creates a georeferenced version using the original .tif file. Next for each cell in the mask a representativae Polygon is created and stored in a geodataframe. These are then merged to make larger shapes. These larger shapes are then stored in a geojson. The script is quite efficient (less than 90s on my macbook) and is multithreaded. The first cell contains all of the necessary functions and the following three cells convert the masks of the three sheets into geojsons.

In [3]:
def calc_boundary_points(origin,x_length,y_length):
    a = (origin[0] - (x_length/2), origin[1] - (y_length/2))
    b = (a[0], a[1] + y_length)
    c = (a[0] + x_length, a[1] + y_length)
    d = (a[0] + x_length, a[1])
    return [a, b, c, d]

def make_geo(origin,x_length,y_length):
    geo = shapely.geometry.Polygon(calc_boundary_points(origin,x_length,y_length))
    return geo

def union(x):
        return unary_union(x)

def georeference_masks(masked_image,original_image,output_path,epsg):
    dataset = rasterio.open(masked_image, 'r')
    original = rasterio.open(original_image, 'r')
    new_dataset = np.clip(dataset.read(1)+dataset.read(2)+dataset.read(3),0,255)
    bands = [1]
    data = np.reshape(new_dataset, (1,) + new_dataset.shape )
    
    crs = {'init': f'epsg:{epsg}'}

    with rasterio.open(output_path, 'w', driver='GTiff',
                    width=data.shape[2], height=data.shape[1],
                    count=1, dtype=data.dtype, nodata=0,
                    transform=original.transform, crs=crs) as dst:
        dst.write(data, indexes=bands)
    return original.get_transform()

def mask_to_shape(geo_mask,transform,n_cores,n_geometries_in_chunk,shape_destination,epsg):
    x_length = abs(transform[1])
    y_length = abs(transform[5])
    print("Reading raster...")
    mask_img = rxr.open_rasterio(geo_mask)
    x, y, mask = mask_img.x.values, mask_img.y.values, mask_img.values
    x, y = np.meshgrid(x, y)
    x, y, mask = x.flatten(), y.flatten(), mask.flatten()

    print("Converting to GeoDataFrame...")
    mask_pd = pd.DataFrame.from_dict({'mask': mask, 'x': x, 'y': y})
    mask_threshold = 0
    mask_pd = mask_pd[mask_pd['mask'] > mask_threshold]
    mask_pd['polygons'] = mask_pd.apply(lambda x: make_geo((x['x'],x['y']),x_length,y_length),axis=1)

    dem_vector = gpd.GeoDataFrame(geometry=gpd.GeoSeries(mask_pd['polygons'], crs=f'EPSG:{epsg}'))
    dem_vector['mask'] = mask_pd['mask']

    # Converting GeoSeries to list of geometries
    geoms = dem_vector['geometry'].tolist() #list(dem_vector)

    # # Converting geometries list to nested list of geometries
    geom_arr = []

    for i in range(0, len(geoms), n_geometries_in_chunk):
        geom_arr.append(geoms[i:i+n_geometries_in_chunk])

    # Creating multiprocessing pool to perform union operation of chunks of geometries
    with mp.Pool(n_cores) as p:
        geom_union = p.map(union, geom_arr)

    # Perform union operation on returned unioned geometries
    total_union = unary_union(geom_union)

    # Creating GeoDataFrame for total_union
    union_vector_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(total_union))

    # Saving GeoDataFrame to shapefile
    union_vector_gdf.to_file(shape_destination, driver='GeoJSON', crs=f'EPSG:{epsg}')
    
def main_shape_create(sheet,epsg,geo_mask_dest,masked_image_path,original_tile,n_cores,chunk_size,shape_dest):
    transform = georeference_masks(masked_image_path, original_tile, geo_mask_dest,epsg)
    mask_to_shape(geo_mask_dest,transform,n_cores,chunk_size,shape_dest,epsg)
    

In [4]:
sheet_official = "1092_1990"
epsg = "21781"
sheet_number = 'sheet1'

geo_mask_dest = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_geomask.tif")
masked_image_path = os.path.join(os.getcwd(),f"QATM/newalpha_result/{sheet_number}_merged.png")
original_tile = f"/Users/jmccarty/Data/220206_ReTo_carto/template_matching/tif_files/LKg_{sheet_official}.tif"
shape_destination = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_maskshape.geojson")

main_shape_create(sheet_official,epsg,geo_mask_dest,masked_image_path,original_tile,n_cores,chunk_size,shape_destination)

/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Reading raster...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/pyproj/crs/_cf1x8.py:511: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Converting to GeoDataFrame...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [5]:
sheet_official = "1111_1988"
epsg = "21781"
sheet_number = 'sheet2'

geo_mask_dest = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_geomask.tif")
masked_image_path = os.path.join(os.getcwd(),f"QATM/newalpha_result/{sheet_number}_merged.png")
original_tile = f"/Users/jmccarty/Data/220206_ReTo_carto/template_matching/tif_files/LKg_{sheet_official}.tif"
shape_destination = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_maskshape.geojson")

main_shape_create(sheet_official,epsg,geo_mask_dest,masked_image_path,original_tile,n_cores,chunk_size,shape_destination)

/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Reading raster...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/pyproj/crs/_cf1x8.py:511: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Converting to GeoDataFrame...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [6]:
sheet_official = "1132_1989"
epsg = "21781"
sheet_number = 'sheet3'

geo_mask_dest = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_geomask.tif")
masked_image_path = os.path.join(os.getcwd(),f"QATM/newalpha_result/{sheet_number}_merged.png")
original_tile = f"/Users/jmccarty/Data/220206_ReTo_carto/template_matching/tif_files/LKg_{sheet_official}.tif"
shape_destination = os.path.join(os.getcwd(),f"QATM/notebooks/shape_tests/LKg_{sheet_official}_maskshape.geojson")

main_shape_create(sheet_official,epsg,geo_mask_dest,masked_image_path,original_tile,n_cores,chunk_size,shape_destination)

/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Reading raster...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/pyproj/crs/_cf1x8.py:511: UserWarning: angle from rectified to skew grid parameter lost in conversion to CF
  warnings.warn(


Converting to GeoDataFrame...


/opt/homebrew/Caskroom/miniforge/base/envs/cartography/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
